In [1]:
from pprint import pprint

from pvsamlab.system import System 

In [2]:
for modules_per_string in range(25, 31):
    print(f"Running simulation for modules_per_string={modules_per_string}")
    plant = System(modules_per_string=modules_per_string)
    pprint(plant.model_results)
    print("-" * 50)

Running simulation for modules_per_string=25
{'ac_annual': 303004440.6349879, 'voc_max': 1286.0765075683591}
--------------------------------------------------
Running simulation for modules_per_string=26
{'ac_annual': 303004665.958397, 'voc_max': 1337.5195678710936}
--------------------------------------------------
Running simulation for modules_per_string=27
{'ac_annual': 303000036.16055524, 'voc_max': 1388.9626281738279}
--------------------------------------------------
Running simulation for modules_per_string=28
{'ac_annual': 302984319.60066444, 'voc_max': 1440.3848388671872}
--------------------------------------------------
Running simulation for modules_per_string=29
{'ac_annual': 302990759.1831903, 'voc_max': 1491.8271545410153}
--------------------------------------------------
Running simulation for modules_per_string=30
{'ac_annual': 302894452.50494397, 'voc_max': 1543.291809082031}
--------------------------------------------------


In [ ]:
from pvsamlab.climate import download_nsrdb_csv

download_nsrdb_csv(
    lat=30.9759,
    lon=-97.2465,
    start_year=2022,
    end_year=2022,
    frequency="hourly",
    time_zone="America/Los_Angeles",
    output_dir=".",
)

In [10]:
chillingham = System(lat=30.9759, lon=-97.2465, met_year='tmy')
print(chillingham.model_results)


{'ac_annual': 259242441.62802073, 'voc_max': 1410.8617919921871}


In [11]:
for year in range(1998, 2024):
    print(f"Running simulation for year={year}")
    plant = System(lat=30.9759, lon=-97.2465, met_year=year)
    pprint(plant.model_results)
    print("-" * 50)

Running simulation for year=1998
{'ac_annual': 265857900.71057835, 'voc_max': 1414.5730224609374}
--------------------------------------------------
Running simulation for year=1999
{'ac_annual': 279223268.454046, 'voc_max': 1424.1638427734372}
--------------------------------------------------
Running simulation for year=2000
{'ac_annual': 265263716.7917375, 'voc_max': 1406.9629150390626}
--------------------------------------------------
Running simulation for year=2001
{'ac_annual': 262276390.9807356, 'voc_max': 1413.8641357421873}
--------------------------------------------------
Running simulation for year=2002
{'ac_annual': 260315043.29980686, 'voc_max': 1428.1669677734374}
--------------------------------------------------
Running simulation for year=2003
{'ac_annual': 252741167.7164493, 'voc_max': 1422.2456787109372}
--------------------------------------------------
Running simulation for year=2004
{'ac_annual': 242414216.975923, 'voc_max': 1421.2240478515625}
---------------

SystemError: <built-in method assign of Pvsamv1 object at 0x112af1380> returned a result with an exception set